# Packages

In [1]:
### Import packages ###
import itertools
import numpy as np
import pandas as pd
import sys
sys.path.append('..')
from utils.Auxiliary import CreateParameterVectorFunction
from utils.Auxiliary import FilterJobNames

---

# Create Parameter Vector

In [2]:
# Data: Iris  MONK1  MONK3  Bar7 (10)  COMPAS (50) | BankNote (10)  BreastCancer (5)  CarEvaluation (10)  FICO (50)  Haberman
Data = "BankNote"
ParameterVector = CreateParameterVectorFunction(Data = Data,
                                  Seed = range(0,1),                     # range(0,50)
                                  RashomonThreshold = 0.025,        # For TreeFarms
                                  DiversityWeight = 0.333,          # For BatchQBC
                                  DensityWeight = 0.333,            # For BatchQBC
                                  BatchSize = 10,                           # 1 5 or 10
                                  Partition = "short",                                # [short, medium, long, largemem, compute, cpu-g2-mem2x]
                                  Time = "00:59:00",                                     # [00:59:00, 11:59:00, 6-23:59:00]
                                  Memory = "30000M",                                   # [100M, 30000M, 100000M]
                                  IncludePL_RF=True,       # Passive Learning with RandomForestClassifierPredictor
                                  IncludePL_GPC=True,      # Passive Learning with GaussianProcessClassifierPredictor
                                  IncludePL_BNN=True,      # Passive Learning with BayesianNeuralNetworkPredictor
                                  IncludeBALD_BNN=True,    # BALD with BayesianNeuralNetworkPredictor
                                  IncludeBALD_GPC=True,    # BALD with GaussianProcessClassifierPredictor
                                  IncludeQBC_TreeFarms_Unique=True, # BatchQBC with TreeFarmsPredictor (UniqueErrorsInput=1)
                                  IncludeQBC_TreeFarms_Duplicate=True, # BatchQBC with TreeFarmsPredictor (UniqueErrorsInput=0)
                                  IncludeQBC_RF=True,      # BatchQBC with RandomForestClassifierPredictor
                                  IncludeLFR_TreeFarms=False, # NEW: For TreefarmsLFRPredictor (requires RefitFrequency)
                                  RefitFrequency=1          # Default refit frequency for LFR (1 = every iter)
                                  )

# Evaluate

In [3]:
ParameterVector["JobName"]

0           0BN_PL_RF_UEI0A0_DW0_DEW0_B10
1          0BN_PL_GPC_UEI0A0_DW0_DEW0_B10
2          0BN_PL_BNN_UEI0A0_DW0_DEW0_B10
3        0BN_BALD_BNN_UEI0A0_DW0_DEW0_B10
4        0BN_BALD_GPC_UEI0A0_DW0_DEW0_B10
5    0BN_UNREAL_UEI1A025_DW333_DEW333_B10
6    0BN_DUREAL_UEI0A025_DW333_DEW333_B10
7      0BN_QBC_RF_UEI0A0_DW333_DEW333_B10
Name: JobName, dtype: object

# Only run these ones

In [4]:
# # ### Simulations that failed ###
# FilterText = ['26B7_UA02']
# ParameterVector = FilterJobNames(ParameterVector,FilterText)
# ParameterVector.index = range(0,len(ParameterVector))
# ParameterVector

---

# Save Parameter Vector

In [5]:
# ### Resave ###
# output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
# ParameterVector.to_csv(output_path, index=False)